# Inventory Allocation

This notebook provides two prototypes of solvers for the inventory allocation problem. We use the problem statements that are essentially equivalent to the classic linear transportation problem.

The first prototype focuses on the basic formulation where a single product need to be allocated across $n$ warehouses to serve $m$ markets. 

The second prototype focuses on an extended formulation of the problem where some product combinations are frequently purhased together. In this formulation, we need to ensure that such combinations (itemsets) are collocated so that orders are not split across multiple warehouses.

In [ ]:
#
# Initialization
#
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np

pywrapinit.CppBridge.InitLogging('linear-transportation-problem.py')
cpp_flags = pywrapinit.CppFlags()
cpp_flags.logtostderr = True
cpp_flags.log_prefix = False
pywrapinit.CppBridge.SetFlags(cpp_flags)

### Scenario 1: Allocation Optimization for a Single Product

We consider a retailer that needs to allocate the inventory across multiple wareshouses. We assume only one SKU. 

We assume $n$ warehouses that serve $m$ markets (demand points). The cost of shipping one unit from $i$-th warehouse to $j$-th market is $c^s_{ij}$. The unit selling price in market $j$ is $p_j$.

We aim to determine the number of units $q_i$ that need to be allocated in each warehouse by solving the following LP problem:
$$
\max_{q} \quad  \sum_j p_j \sum_i q_{ij} - \sum_i\sum_j c^s_{ij} q_{ij}
$$

subject to

$$
\sum_{i} q_{ij} \le D_j, \quad j=1,\ldots,m
$$

where $q_{ij}$ if the number of units allocated for market $j$ in warehouse $i$ and $D_j$ is the demand in market $j$. The allocation level at warehouse $i$ is then
$$
q_i = \sum_j q_{ij}
$$


In [4]:
solver = pywraplp.Solver.CreateSolver('GLOP')

# Number of warehouses (index i)
n = 3 

# Number of markets (index j)
m = 5

# Market demands
d = [10, 20, 30, 30, 20]

# Shipping costs (cost of shipping one unit from warehouse i to market j)
cs = np.array([
    [1.0, 1.0, 1.0, 1.0, 0.2],
    [1.0, 1.0, 0.5, 0.3, 0.5],
    [0.5, 0.3, 3.0, 0.5, 0.5]
])

# Market prices
p = [1, 2, 1, 3, 1]


#
# Define variables
#
q = np.array([[solver.NumVar(0.0, solver.infinity(), f'q{i}{j}') for j in range(m)] for i in range(n)])
print('Number of variables =', solver.NumVariables())


#
# Define constraints
#
for j in range(m):
    solver.Add(np.sum(q[:, j]) <= d[j])
print('Number of constraints =', solver.NumConstraints())

#
# Define the objective
#
revenue = np.sum( [ p[j] * np.sum(q[:, j]) for j in range(m) ])
shipping_costs = np.sum( cs * q )
solver.Maximize(revenue - shipping_costs)

#
# Solve 
#
status = solver.Solve()

#
# Print the results
#
solution_value = np.vectorize(lambda x: x.solution_value())
    
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

    for i in range(n):
        q_i = np.sum(solution_value(q[i, :]))
        print(f'Stock level at warehouse {i}:  {q_i}')
else:
    print('The problem does not have an optimal solution.')

Number of variables = 15
Number of constraints = 5
Solution:
Objective value = 151.0
Stock level at warehouse 0:  20.0
Stock level at warehouse 1:  60.0
Stock level at warehouse 2:  30.0


### Scenario 2: Frequent Itemsets Should Be Collocated

In this section, we consider an extended formulation of the problem with multiple SKUs. We can specify SKU combinations (itemsets) that are frequently purhased in one order, and the solver is searhing for allocation that ensures that each order can be sourced from one warehouse.    

In [44]:
# Number of warehouses
W = 3 

# Number of markets
M = 5

# Demand by itemset and market
d = [
    ({'a'},           [10, 20, 30, 30, 20]), # orders with product a only
    ({'b'},           [20, 30, 10, 40, 10]), # orders with product b only
    ({'c'},           [40, 30, 10, 10, 20]), # orders with product c only
    ({'a', 'b'},      [5, 7, 3, 3, 8]),      # orders with a and b
    ({'b', 'c'},      [3, 9, 2, 1, 4]),      # orders with b and c
    ({'a', 'c'},      [7, 2, 5, 9, 7]),      # orders with a and c
    ({'a', 'b', 'c'}, [1, 0, 2, 1, 0])       # orders with a, b, and c
]

# Number of itemsets
S = len(d)

# Shipping costs (cost of shipping one unit from warehouse w to market m)
cs = np.array([
    [1.0, 1.0, 1.0, 1.0, 0.2],
    [1.0, 1.0, 0.5, 0.3, 0.5],
    [0.5, 0.3, 3.0, 0.5, 0.5]
])

# Product unit price by market
pr = {
    'a': [1, 2, 1, 3, 1], # product a
    'b': [2, 1, 1, 1, 2], # product b
    'c': [1, 2, 1, 1, 1]  # product c
}

#
# Compute the derived parameters
#
dv = np.array([v for (k,v) in d])

# Compute prices for all itemsets
pr_itemset = np.zeros((S, M))
for s in range(S):
    for m in range(M):
        items = d[s][0]
        for item in items:
            pr_itemset[s, m] += pr[item][m]
        
#
# Specify and solve the optimization problem
#
solver = pywraplp.Solver.CreateSolver('GLOP')

#
# Define variables
#
q = np.array([[[solver.NumVar(0.0, solver.infinity(), f'q{s}{w}{m}') 
                for m in range(M)] 
               for w in range(W)] 
              for s in range(S)])
print('Number of variables =', solver.NumVariables())


#
# Define constraints
#
for m in range(M): 
    for s in range(S):
        solver.Add(np.sum(q[s, :, m]) <= dv[s, m])
print('Number of constraints =', solver.NumConstraints())


#
# Define the objective
#
revenue = np.sum( [[ pr_itemset[s, m] * np.sum(q[s, :, m]) for m in range(M) ] for s in range(S)])
shipping_costs = np.sum( cs * np.sum(q, axis = 0) )
solver.Maximize(revenue - shipping_costs)

#
# Solve 
#
status = solver.Solve()

#
# Print the results
#
solution_value = np.vectorize(lambda x: x.solution_value())
    
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    
    q_sw = np.zeros((S, W))
    for s in range(S):
        for w in range(W):
            q_sw[s, w] = np.sum(solution_value(q[s, w, :]))
    
    q_pw = {p : np.zeros(W) for p in pr.keys()}
    for w in range(W):
        for s in range(S):
            for i in d[s][0]:
                q_pw[i][w] += q_sw[s, w]
            
    for w in range(W):
        print(f'\nWarehouse {w}:')
        for k,v in q_pw.items():
            print(f'\tProduct {k} : {v[w]} units')
        
        print(f'\tBreakdown by itemsets:')
        for s in range(S):
            print(f'\t\t{d[s][0]} : {q_sw[s, w]}')
            
else:
    print('The problem does not have an optimal solution.')

Number of variables = 105
Number of constraints = 35
Solution:
Objective value = 553.6

Warehouse 0:
	Product a : 35.0 units
	Product b : 22.0 units
	Product c : 31.0 units
	Breakdown by itemsets:
		{'a'} : 20.0
		{'b'} : 10.0
		{'c'} : 20.0
		{'a', 'b'} : 8.0
		{'c', 'b'} : 4.0
		{'a', 'c'} : 7.0
		{'a', 'b', 'c'} : 0.0

Warehouse 1:
	Product a : 83.0 units
	Product b : 62.0 units
	Product c : 40.0 units
	Breakdown by itemsets:
		{'a'} : 60.0
		{'b'} : 50.0
		{'c'} : 20.0
		{'a', 'b'} : 6.0
		{'c', 'b'} : 3.0
		{'a', 'c'} : 14.0
		{'a', 'b', 'c'} : 3.0

Warehouse 2:
	Product a : 52.0 units
	Product b : 75.0 units
	Product c : 92.0 units
	Breakdown by itemsets:
		{'a'} : 30.0
		{'b'} : 50.0
		{'c'} : 70.0
		{'a', 'b'} : 12.0
		{'c', 'b'} : 12.0
		{'a', 'c'} : 9.0
		{'a', 'b', 'c'} : 1.0
